In [45]:
import numpy as np
import pandas as pd
import os


In [46]:
import torch
import torch.nn as nn

In [69]:
data_dir = '/opt/ml/level2_dkt_recsys-level2-recsys-10-2/data' # 경로는 상황에 맞춰서 수정해주세요!
csv_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
df = pd.read_csv(csv_file_path) 

In [70]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


In [71]:
ass_aver_ = df.groupby(['assessmentItemID'])['answerCode'].agg(['mean'])
df = pd.merge(df, ass_aver_, on=['assessmentItemID'], how="left")
df = df.rename(columns={'mean':'ass_aver'})
df['ass_aver'] = df['ass_aver'].apply(x_100)
user_aver_ = df.groupby(['userID'])['answerCode'].agg(['mean'])
df = pd.merge(df, user_aver_, on=['userID'], how="left")
df = df.rename(columns={'mean':'user_aver'})
df['user_aver'] = df['user_aver'].apply(x_100)
df["big_"] = df["assessmentItemID"].str[2]
big_ = df.groupby(['big_'])['answerCode'].agg(['mean'])
df = pd.merge(df, big_, on=['big_'], how="left")
df = df.rename(columns={'mean':'big'})
df['big'] = df['big'].apply(x_100)
df['shift'] = df.groupby('userID')['answerCode'].shift().fillna(0)
df['past_correct'] = df.groupby('userID')['shift'].cumsum()
df['same_item_cnt'] = df.groupby(['userID', 'assessmentItemID']).cumcount() + 1
df['problem_id'] = df['assessmentItemID'].str[-3:]
problem_id_mean_ = df.groupby('problem_id')['answerCode'].agg(['mean'])
df = pd.merge(df, problem_id_mean_, on=['problem_id'], how="left")
df = df.rename(columns={'mean':'problem_id_mean'})
df['problem_id_mean'] = df['problem_id_mean'].apply(x_100)
df['month'] = df['Timestamp'].str[5:7]
df['month'].astype(int)
month_mean_ = df.groupby('month')['answerCode'].agg(['mean'])
df = pd.merge(df, month_mean_, on=['month'], how="left")
df = df.rename(columns={'mean':'month_mean'})
df['month_mean'] = df['month_mean'].apply(x_100)

In [51]:
def x_100(value):   #0~100범위로 바꿔주기
    return int(value * 100)

In [63]:
df['month'] = df['Timestamp'].str[5:7]
df['month'].astype(int)
month_mean = df.groupby('month')['answerCode'].agg(['mean'])
df = pd.merge(df, month_mean, on=['month'], how="left")
df = df.rename(columns={'mean':'month_mean'})
df['month_mean'] = df['month_mean'].apply(x_100)

In [72]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,ass_aver,user_aver,big_,big,shift,past_correct,same_item_cnt,problem_id,problem_id_mean,month,month_mean
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,98,63,6,70,0.0,0.0,1,001,74,03,68
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,96,63,6,70,1.0,1.0,1,002,72,03,68
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,91,63,6,70,1.0,2.0,1,003,68,03,68
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,96,63,6,70,1.0,3.0,1,004,66,03,68
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,94,63,6,70,1.0,4.0,1,005,59,03,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,43,55,3,70,0.0,1.0,1,005,59,06,65
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,64,55,4,68,0.0,1.0,1,001,74,08,65
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,63,55,4,68,1.0,2.0,1,002,72,08,65
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,79,55,4,68,1.0,3.0,1,003,68,08,65


In [53]:
df['month_mean']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
2266581   NaN
2266582   NaN
2266583   NaN
2266584   NaN
2266585   NaN
Name: month_mean, Length: 2266586, dtype: float64

In [18]:
df['big'] = df['assessmentItemID'].str[2]

In [20]:
df.groupby('big')['answerCode'].agg(['mean'])

,mean
big,
1,0.800876
2,0.737593
3,0.702238
4,0.684056
5,0.658208
6,0.709232
7,0.521876
8,0.502598
9,0.449948


In [21]:
df["big"] = df["assessmentItemID"].str[2]
big = df.groupby(['big'])['answerCode'].agg(['mean'])
df = pd.merge(df, big, on=['big'], how="left")
df = df.rename(columns={'mean':'big'})
# df['big'] = df['big'].apply(self.x_100)

In [25]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,big,big,assessment_mean,assessment_sum
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,6,0.709232,0.982063,219
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,6,0.709232,0.964126,215
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,6,0.709232,0.910314,203
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,6,0.709232,0.968610,216
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,6,0.709232,0.941704,210
...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,3,0.702238,0.435252,121
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,4,0.684056,0.649635,178
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,4,0.684056,0.635036,174
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,4,0.684056,0.791971,217


In [24]:
correct = df.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum'])
df.columns = ["assessment_mean", 'assessment_sum']

df = pd.merge(df, correct_a, on=['assessmentItemID'], how="left")

In [28]:
import datetime

In [27]:
df['Timestamp']

0          2020-03-24 00:17:11
1          2020-03-24 00:17:14
2          2020-03-24 00:17:22
3          2020-03-24 00:17:29
4          2020-03-24 00:17:36
                  ...         
2266581    2020-06-05 06:50:21
2266582    2020-08-21 01:06:39
2266583    2020-08-21 01:06:50
2266584    2020-08-21 01:07:36
2266585    2020-08-21 01:08:49
Name: Timestamp, Length: 2266586, dtype: object

In [40]:
df['month'] = df['Timestamp'].str[5:7]
df['month'].astype(int)
df['month_mean'] = df.groupby('month')['answerCode'].agg(['mean'])

0          3
1          3
2          3
3          3
4          3
          ..
2266581    6
2266582    8
2266583    8
2266584    8
2266585    8
Name: month, Length: 2266586, dtype: int64

In [42]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,big,big,assessment_mean,assessment_sum,day,month
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,6,0.709232,0.982063,219,03,03
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,6,0.709232,0.964126,215,03,03
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,6,0.709232,0.910314,203,03,03
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,6,0.709232,0.968610,216,03,03
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,6,0.709232,0.941704,210,03,03
...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,3,0.702238,0.435252,121,06,06
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,4,0.684056,0.649635,178,08,08
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,4,0.684056,0.635036,174,08,08
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,4,0.684056,0.791971,217,08,08


In [41]:
df['month_mean'] = df.groupby('month')['answerCode'].agg(['mean'])

,mean
month,
01,0.712199
02,0.687222
03,0.682643
04,0.649545
05,0.643317
06,0.651967
07,0.665650
08,0.657599
09,0.668872


In [36]:
df['day'].astype(int)

0          3
1          3
2          3
3          3
4          3
          ..
2266581    6
2266582    8
2266583    8
2266584    8
2266585    8
Name: day, Length: 2266586, dtype: int64

In [44]:
print('잘 나와유..')

잘 나와유..


In [16]:
(64 // 4) * 6, 64

(96, 64)